In [1]:
AUTHOR_NAME = 'Jane Deijnen'
AUTHOR_ID_NR = '1354396'
AUTHOR_DATE = '2020-05-05'

AUTHOR_NAME, AUTHOR_ID_NR, AUTHOR_DATE

('Jane Deijnen', '1354396', '2020-05-05')

In [2]:
import numpy as np
import pandas as pd
import bokeh as bk

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import seaborn as sns
sns.set()  # set Seaborn defaults
plt.rcParams['figure.figsize'] = 10, 5  # default hor./vert. size of plots, in inches
plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after import seaborn
from sklearn.cluster import KMeans  # for clustering

from bokeh.io import output_notebook, show, reset_output, curdoc
from bokeh.models import Slider, CustomJS
from bokeh.plotting import figure
from bokeh.layouts import layout, column
from bokeh.models import (
    ColumnDataSource, Div,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
    FactorRange
)
from bokeh.palettes import BuPu
from bokeh.palettes import Spectral6
output_notebook()

Loading BokehJS ...

In [3]:
eyetracking_data = pd.read_csv('metro_data.csv', encoding = 'latin1', sep = ";")
eyetracking_data.head()

,Timestamp,StimuliName,FixationIndex,FixationDuration,MappedFixationPointX,MappedFixationPointY,user,description
0,2586,01_Antwerpen_S1.jpg,9,250,1151,458,p1,color
1,2836,01_Antwerpen_S1.jpg,10,150,1371,316,p1,color
2,2986,01_Antwerpen_S1.jpg,11,283,1342,287,p1,color
3,3269,01_Antwerpen_S1.jpg,12,433,762,303,p1,color
4,3702,01_Antwerpen_S1.jpg,13,183,624,297,p1,color


In [4]:
#dataframe of Antwerper_s1 stimulus only
Antwerpen_s1 = eyetracking_data[eyetracking_data['StimuliName'] == '01_Antwerpen_S1.jpg']
Antwerpen_s1.head()

,Timestamp,StimuliName,FixationIndex,FixationDuration,MappedFixationPointX,MappedFixationPointY,user,description
0,2586,01_Antwerpen_S1.jpg,9,250,1151,458,p1,color
1,2836,01_Antwerpen_S1.jpg,10,150,1371,316,p1,color
2,2986,01_Antwerpen_S1.jpg,11,283,1342,287,p1,color
3,3269,01_Antwerpen_S1.jpg,12,433,762,303,p1,color
4,3702,01_Antwerpen_S1.jpg,13,183,624,297,p1,color


In [5]:
#Bokeh transition matrix (Can't get the axes how I want it)
#I get errors when running on the server. Idk what is wrong, but this whole graph is weird tbh. Idk how to fix it
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"
TOOLTIPS=[("Value", "@value"),
         ("AOI transition from", "@x"),
         ("to", "@y")]

slider_cluster = Slider(title="Amount of Clusters", start=1, end=8, value=3, step=1)

colormap = cm.get_cmap("BuPu")
bokehpalette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]
mapper = LinearColorMapper(palette=bokehpalette, low=0.0, high=1.0)

source =  ColumnDataSource(data=dict(x=[], y=[], value=[]))

z = figure(title="Transition Matrix", toolbar_location='below',
           toolbar_sticky=False, tools=TOOLS, tooltips=TOOLTIPS)
z.rect(x='x', y='y', width=1, height=1, source=source,
       fill_color={'field': 'value', 'transform': mapper}, line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=8),
                     label_standoff=6, border_line_color=None, location=(0, 0))
z.add_layout(color_bar, 'right')
z.xaxis.axis_label = 'AOI'
z.yaxis.axis_label = 'AOI'

show(z)

In [6]:
def select_graphs():
    n_clusters = slider_cluster.value
    X_km = Antwerpen_s1[['MappedFixationPointX', 'MappedFixationPointY']].copy()
    km = KMeans(n_clusters)
    km.fit(X_km)
    centers = pd.DataFrame(km.cluster_centers_, columns=X_km.columns)
    X_km['cluster'] = km.labels_

    user = Antwerpen_s1["user"]
    X_km = X_km.join(user)
    X_km = X_km.reset_index()
    
    matrix = []
    AOI = []
    count = 0
    for i in range (0, n_clusters):
        matrix.append([])
        AOI.append(count+1)
        count += 1
    for i in range (0, n_clusters):
        for j in range (0, n_clusters):
            matrix[i].append(j)
            matrix[i][j] = 0
            
    cluster = X_km.loc[0, 'cluster']
    user = X_km.loc[0, 'user']
    for n in range(1, X_km.index[-1]+1):
        cluster_compare = X_km.loc[n, 'cluster']
        user_compare = X_km.loc[n, 'user']
        if cluster != cluster_compare and user == user_compare: #I think this solves the problem with the increment between users
            matrix[cluster][cluster_compare] = matrix[cluster][cluster_compare]+1
        cluster = cluster_compare
        user = user_compare
    
    matrix = np.array(matrix)
    
    m = np.amax(matrix)
    norm_matrix = (1/m) * matrix
    df_norm_matrix = pd.DataFrame(norm_matrix, index=AOI, columns=AOI)
    
    matrix_r = df_norm_matrix.reset_index()
    matrix_rows = pd.melt(matrix_r, id_vars=['index'], value_vars=AOI, var_name='target_AOI')

    return matrix_rows

In [7]:
def update():
    matrix_rows = select_graphs()
    #source.data = ColumnDataSource(matrix_rows).data
    x = list(matrix_rows['index'])
    y = list(matrix_rows['target_AOI'])
    value = list(matrix_rows['value'])
    source.data = dict(
        x=x,
        y=y,
        value=value
    )

In [8]:
#execution
controls = [slider_cluster]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

inputs = column(*controls, width=320, height=1000)
inputs.sizing_mode = "fixed"
l = layout([
    [inputs, z],
], sizing_mode="scale_both")

update()  # initial load of the data

curdoc().add_root(l)
curdoc().title = "Graph"

In [9]:
show(l)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

